# Practical intro to Quantum Chemistry

## with Orca!

[![](images/orca.png)](https://orcaforum.cec.mpg.de/)


### Requirements / Bare-bones:

* [**Orca 3.0.3**](https://orcaforum.cec.mpg.de/).
* Python 2 or 3, with [Ipython/Jupyter](https://jupyter.org/) notebook abilities. 
* Python modules: *matplotlib, imolecule, openbabel*.

### Recomended / Good stuff to check out
 
* [**Orca Manual**](https://orcaforum.cec.mpg.de/OrcaManual.pdf).
* [**Orca Input Library**](https://sites.google.com/site/orcainputlibrary/).
* A sweet sweet text editor, I suggest [**Sublime Text**](http://www.sublimetext.com/).
* Molecular visualizer, [**Avogadro**](http://avogadro.cc/wiki/Main_Page) is good and easy and [**VMD**](http://www.ks.uiuc.edu/Research/vmd/) is powerful.

## Quantum Chemistry

Quantum chemistry is a mix of physics, chemistry, and applied mathematics and computation.

In Quantum Chemistry try to solve either the time-independent Schrödinger equation (stationary states)
to get molecular properties:
$$
H | \psi \rangle = E | \psi \rangle,
$$
or time-dependent:
$$
i\hbar \frac{\partial \Psi}{\partial t} = -\frac{\hbar^2}{2m}
\frac{\partial^2 \Psi}{\partial x^2} + V \Psi
$$

Relevant quotes:

> “We can calculate everything"
> - Enrico Clementi


## Input files

Almost all quantum chemistry packages work with input files, which are text files with a few key words that indicate which model we will use for a calculation.

Here is an example input file


In [134]:
!head input.inp

! CCSD cc-pVTZ Opt
! NUMGRAD

*xyzfile 0 1 mol.xyz



### Some questions that arise when doing calculations:

* What software should I use?
* Is my algorithm parallelized?
* Which geometry?
* Which methodology should I use?
* DFT or Wavefunction methods? or Quantum Monte Carlo?
* Open or Closed Shell?
* Multireference wavefunction?
* How accurate and how much can I trust my calculations?
* How do I treat a solvated molecule? Which solvation model?
* Do I need to incorporate dispersion effects?
* Which basis set should I use?
* Polarization, Diffusion functions?
* Spin coupling effects?

Most decisions are coupled so you really can't test each independently.

Like a fellow quantum chemist said, sometimes you just got to do science and just marry whichever of what works best for you!

## The Quantum Chemistry Panorama (stationary)
### So what does this all mean?
![](images/overview.png)

#### The main problem is treating electron-electron interactions.

### To summarize:

* **DFT:** Based on charge density, a physical measurable, pretty cheap $\mathcal{O}(n^3)-\mathcal{O}(n^4)$ and accurate, not 100% reliable. You have to decide on a functional and a basis set. Also **the Hohenberg-Kohn theorems** state that any molecular property can be derived from the charge density and there exists a universal functional which will solve all your problems.
* **Wavefunction:** Pretty expensive $\mathcal{O}(n^3)-\mathcal{O}(n!)$, but reliable and there is a clear idea of how to systematically improve your solution and to arrive to the exact theoretical limit. Sometimes you can 

* **Semi-empirical:** Relies on force fields, parameterizations, model-fitting and other neat tricks to get cheap and semi accurate results for a fraction of the cost. 


## First is First
### Choose you favorite molecule!*
#### * of 10 electrons or less

### XYZ geometry files
Contains basic 3d coordinates of a molecule

1. Number of atoms
2. Comment line, can be whatever
3. Atom # 1, Element X Y Z

... follow till N atoms

**Now** make a xyz file, name it **mol.xyz**, you will use this molecule for your exploration

In [55]:
!head mol.xyz

3

H          0.63109       -0.02651        0.47485
O          0.14793        0.02998       -0.34219
H         -0.77901       -0.00348       -0.13266


In [133]:
import imolecule
imolecule.draw("mol.xyz")

## Run a calculation
Assuming you installed Orca correctly, you can run your calculations as show below via the command:
```bash
orca inputfile > outputfile
```
should take only a few seconds..

In [ ]:
!orca input.inp > output.out

let's check out our output file

In [43]:
!tail -50 output.out

Electron density file                           ... input.scfp.tmp

-------------
DIPOLE MOMENT
-------------
                                X             Y             Z
Electronic contribution:      0.07182       0.01456      -0.16614
Nuclear contribution   :     -0.38844      -0.07874       0.89861
                        -----------------------------------------
Total Dipole Moment    :     -0.31662      -0.06418       0.73246
                        -----------------------------------------
Magnitude (a.u.)       :      0.80054
Magnitude (Debye)      :      2.03482


                           *** MDCI DENSITY ***

------------------------------------------------------------------------------
                       ORCA ELECTRIC PROPERTIES CALCULATION
------------------------------------------------------------------------------

Dipole Moment Calculation                       ... on
Quadrupole Moment Calculation                   ... off
Polarizability Calculation               

Output files are large, so we normally use the magic of programming to extract the information:
#### Grep
Via command line you can use [**grep**](http://www.tecmint.com/12-practical-examples-of-linux-grep-command/) (globally search a regular expression and print), you use like this:
```shell
grep string file
```
Care about energy?

In [47]:
!grep 'FINAL SINGLE POINT ENERGY' output.out

FINAL SINGLE POINT ENERGY       -76.324170464170
FINAL SINGLE POINT ENERGY       -76.324453056742
FINAL SINGLE POINT ENERGY       -76.324556475896
FINAL SINGLE POINT ENERGY       -76.324556533388


time?

In [50]:
!grep 'TOTAL RUN TIME:' output.out

TOTAL RUN TIME: 0 days 0 hours 2 minutes 18 seconds 779 msec


### Better yet, some Python magic
For now a poor man's grep

In [65]:
def findStrings(afile,substr):
    lines=[]
    with open(afile) as astr:
        for line in astr:
            if substr in line:
                 lines.append(line)
    return lines

In [66]:
findStrings('output.out','FINAL SINGLE POINT ENERGY')

['FINAL SINGLE POINT ENERGY       -76.324170464170\n',
 'FINAL SINGLE POINT ENERGY       -76.324453056742\n',
 'FINAL SINGLE POINT ENERGY       -76.324556475896\n',
 'FINAL SINGLE POINT ENERGY       -76.324556533388\n']

In [84]:
def findFinalEnergy(afile):
    lines=[]
    substr='FINAL SINGLE POINT ENERGY'
    with open(afile) as astr:
        for line in astr:
            if substr in line:
                tmpline=line.replace(substr,'').strip()
                lines.append(float(tmpline))
    return lines[-1]

In [86]:
findFinalEnergy('output.out')

-76.324556533388

#### Now computing time

In [108]:
def findComputeTime(afile):
    timestr=''
    substr='TOTAL RUN TIME:'
    milisecs=0
    # find string with time
    with open(afile) as astr:
        for line in astr:
            if substr in line:
                timestr=line.replace(substr,'').strip()
    # there is probably a better way of doing this
    timestr=timestr.split()
    milisecs+=int(timestr[-2]) #milisecs
    milisecs+=1000*int(timestr[-4]) #seconds
    milisecs+=60000*int(timestr[-6]) #minutes
    milisecs+=60*60*1000*int(timestr[-8]) #hours
    milisecs+=24*60*60*1000*int(timestr[-10]) #days

    return milisecs

In [111]:
findComputeTime('output.out')

138779

### Also CCLIB
From the website: http://cclib.github.io/
>cclib is an open source library, written in Python, for parsing and interpreting the results of computational chemistry packages.
>

## Wavefunction methods
### Now to compute!


We are interested in time, accuracy and cost.

Explore the following methods:
* PM3
* Hartree Fock
* MP2
* CCSD
* CCSD(T)
* DLPNO-CCSD
Use the basis set **cc-pVDZ**

And save the energy and times in a dictionary

In [135]:
from collections import OrderedDict
energy = OrderedDict()
time = OrderedDict()

## One last trick!
### Template text replacing for automatic input creation
#### and calculation

1. First we define a list of varying parameters within a input file
2. Define a replace function, which takes a template text file and replaces all instances into a new file
3. Iterate over each variable, create input and calculate!



In [137]:
methods=['PM3','HF','MP2','MP4','CCSD','CCSD(T)','DLPNO-CCSD']

In [138]:
def replace_all(astr, bstr, dict):
    with open(astr, 'r') as afile:
        with open(bstr, 'w') as bfile:
            data = afile.read()
            for key, value in dict.items():
                data = data.replace(key, str(value))
            bfile.write(data)
    return

In [141]:
import os

for method in methods:
    replaceDict={"#{METHOD}":method}
    replace_all('template_method.inp','input.inp', replaceDict)
    os.system('orca input.inp > %s.out'%method)

# Whoa!
## you just calculated 50 years of quantum chemistry in a few minutes!
### now plot energy and time for each method

## DFT functionals

DFT functionals more or less go across what is called Jacob's ladder:
![](images/jacobladderextra.png)

### Now try one function of each class:

* Local
* Gradient corrected (GGA)
* Meta GGA
* Hybrid or hybrid meta GGA
* Perturbatively corrected double hybrid

Plot compute time and energy

## extra: Basis set effects
### with wavefunctions

Compute using your favorite wavefunction method the following basis set:
* cc-pVDZ
* cc-pVTZ
* cc-pVQZ
* cc-pV5Z

### with DFT
Compute using your favorite functional method the following basis set:
* Def2-SV
* Def2-QDVP
* Def2-TZV
* Def2-QZVP

## finally..
So now you know how to do a massive number of calculations, It might feel a bit mechanistic sometimes.. so it's good to remember
> 
"..Computers don‘t solve problems - people do!“
> - Ernest R Davidson
